In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
#создаем общий датафрейм
df_movies = pd.read_csv(r'C:\\Data\\Rec_Sys_Cinema_App\\DS14-1-develop\datasets\\tmdb_5000_movies.csv')
df_credits = pd.read_csv(r'C:\\Data\\Rec_Sys_Cinema_App\\DS14-1-develop\datasets\\tmdb_5000_credits.csv')

df_merge = pd.merge(df_credits, df_movies, left_on='movie_id', right_on='id').drop('id', axis=1)
df_merge.head(3)

,movie_id,title_x,cast,crew,budget,genres,homepage,keywords,original_language,original_title,...,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title_y,vote_average,vote_count
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,...,"[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [6]:
#Фильмы в релизе
df = df_merge.loc[df_merge['status'] == 'Released'].copy()
# удаление пустых строк
df.dropna(axis = 'index', subset = ['overview', 'genres', 'keywords'], inplace = True)
# результат
print(f"Количество фильмов в выборке: {len(df)}")

Количество фильмов в выборке: 4792


In [8]:
#замена пустых значений
df['overview'] = df['overview'].fillna('')
df['keywords'] = df['keywords'].fillna('')

In [9]:
#удаление стоп слов и создание матрицы
tf_idf_vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)

tf_idf_matrix = tf_idf_vectorizer.fit_transform(df['overview'] + ' ' + df['keywords'])

print(f"Размер матрицы составил: {tf_idf_matrix.shape}.")

Размер матрицы составил: (4792, 10000).


In [10]:
#расчет косинусного сходства
cosine_sim = cosine_similarity(tf_idf_matrix, tf_idf_matrix)
#создание датафрейма
sim_cos_df = pd.DataFrame(cosine_sim, index=df['movie_id'], columns=df['movie_id'])
# сохранение в файл 'distance.csv'
sim_cos_df.to_csv('../assets/distance.csv')
sim_cos_df.sample(2)

movie_id,19995,285,206647,49026,49529,559,38757,99861,767,209112,...,182291,286939,124606,14337,67238,9367,72766,231617,126186,25975
movie_id,,,,,,,,,,,,,,,,,,,,,
1487,0.114035,0.099052,0.065557,0.125064,0.089975,0.106626,0.091789,0.108788,0.090192,0.085712,...,0.085383,0.0,0.029907,0.070008,0.026600,0.027322,0.0,0.059678,0.0,0.025472
10375,0.058712,0.055496,0.041788,0.052458,0.050121,0.047549,0.036579,0.031781,0.033862,0.034305,...,0.051531,0.0,0.025153,0.049291,0.023441,0.052318,0.0,0.024946,0.0,0.034775


In [11]:
#сохранение самого датасета
df.rename(columns={'title_x': 'title'}, inplace=True)
df.to_csv('../assets/movies.csv', index=False)
df.sample(5)

,movie_id,title,cast,crew,budget,genres,homepage,keywords,original_language,original_title,...,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title_y,vote_average,vote_count
4756,20981,The Call of Cthulhu,"[{""cast_id"": 4, ""character"": ""The Man"", ""credi...","[{""credit_id"": ""52fe4407c3a368484e00b4a5"", ""de...",50000,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 53, ""nam...",http://www.cthulhulives.org/cocmovie/index.html,"[{""id"": 1523, ""name"": ""obsession""}, {""id"": 303...",en,The Call of Cthulhu,...,"[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}]",2005-01-01,0,47.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,The Call of Cthulhu,6.9,41
3096,91076,Illuminata,"[{""cast_id"": 10, ""character"": ""Tuccio"", ""credi...","[{""credit_id"": ""52fe48c39251416c750b276f"", ""de...",0,"[{""id"": 18, ""name"": ""Drama""}]",NaN,"[{""id"": 10965, ""name"": ""playwright""}, {""id"": 1...",en,Illuminata,...,"[{""iso_3166_1"": ""ES"", ""name"": ""Spain""}, {""iso_...",1998-05-21,0,119.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,Illuminata,6.3,7
4579,762,Monty Python and the Holy Grail,"[{""cast_id"": 11, ""character"": ""King Arthur / V...","[{""credit_id"": ""52fe4272c3a36847f801f66f"", ""de...",400000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 35, ""...",http://www.sonypictures.com/cthe/montypython/,"[{""id"": 113, ""name"": ""holy grail""}, {""id"": 345...",en,Monty Python and the Holy Grail,...,"[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}]",1975-03-13,5028948,91.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,And now! At Last! Another film completely diff...,Monty Python and the Holy Grail,7.8,1708
594,3132,Bad Company,"[{""cast_id"": 12, ""character"": ""Officer Oakes"",...","[{""credit_id"": ""52fe4389c3a36847f805bcfb"", ""de...",70000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",NaN,"[{""id"": 502, ""name"": ""ambush""}, {""id"": 591, ""n...",en,Bad Company,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2002-06-07,65977295,116.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Two Mismatched Partners. One Messed Up Case!,Bad Company,5.4,228
2682,8291,Poetic Justice,"[{""cast_id"": 1, ""character"": ""Justice"", ""credi...","[{""credit_id"": ""52fe449dc3a36847f80a044d"", ""de...",14000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 10749, ""n...",http://www.janetjackson.com,"[{""id"": 1014, ""name"": ""loss of lover""}, {""id"":...",en,Poetic Justice,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1993-07-23,27515786,109.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,A Street Romance.,Poetic Justice,6.9,47
